In [2]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = "notebook_connected"

# import matplotlib.pyplot as plt
# import matplotlib as mpl

# %matplotlib inline

# mpl.rc('font',**{'family':'serif','serif':['Times']})

# Import data & deal with dates
data = pd.read_csv('GSPC.csv', index_col='Date')
data.index = pd.to_datetime(data.index, format="%Y-%m-%d")

data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0


In [5]:
# Generate Returns
returns = data['Adj Close'].pct_change()
fig = go.Figure()
fig.add_trace(go.Scatter(x=returns.index, y=returns.values))
fig.update_layout(template='plotly_white',
                  title='Returns')
fig = fig.show()

In [6]:
# Include or exculde great crash period
post_1929 = False
if post_1929:
    returns = data['Adj Close'].pct_change()
    returns = returns['1940':]
else:
    returns = data['Adj Close'].pct_change()

In [7]:
# Find worst returning period for given length

# INPUT
period_years = 5



period_length = 253*period_years
minimum = 900000000
maximum = -1
accumulations = []
for i in range(len(returns)-period_length):
    temp_returns = returns[i:period_length+i]
    # Create wealth index
    wealth_index = 1000*(1+temp_returns).cumprod()
    # Update min and max if necessary
    if wealth_index[-1] < minimum:
        minimum = wealth_index[-1]
        min_idx = i
    elif wealth_index[-1] > maximum:
        maximum = wealth_index[-1]
        max_idx = i
    accumulations.append(wealth_index[-1])

In [10]:
# Plot Biggest Losing Period
temp_returns = returns[max_idx:period_length+max_idx]
wealth_index = 1000*(1+temp_returns).cumprod()
title_string = 'Best Performing ' + str(period_years) + ' Year Period \n \$1000 Investment in S&P 500'
if post_1929:
    title_string = 'Best Performing ' + str(period_years) + ' Year Period Post-1929 Crash \n \$1000 Investment in S&P 500'

In [31]:
total_return = round((wealth_index[-1]-1000)/1000,2)
print("Final Value : ", round(wealth_index[-1],2))
print("Percentage Change : ", total_return*100, "%")
print("Annualised : ", round(((1+total_return)**(1/period_years)-1)*100,2), "%")
print("Period : ", wealth_index.index[0], " - ", wealth_index.index[-1])
print(f'Length of Period: {(wealth_index.index[-1]-wealth_index.index[0]).days/365 :.2f} years')

fig = go.Figure()
fig.add_trace(go.Scatter(x=wealth_index.index,
                         y=wealth_index.values))
fig.update_layout(template='plotly_white',
                  title='Accumulated Value of $1000 Investment in S&P500',
                  height=500,
                  width=1300)
fig = fig.show()

Final Value :  3870.75
Percentage Change :  287.0 %
Annualised :  31.08 %
Period :  1932-07-11 00:00:00  -  1937-08-09 00:00:00
Length of Period: 5.08 years
